# 01. 정확도(Accuracy)

<br>

# 1. 정확도(Accuracy)

- 실제 데이터에서 예측 데이터가 얼마나 같은 지를 판단

$
\quad
정확도(Accuracy) = {예측\,결과가\,동일한\,데이터\,건수 \over 전체\,예측\,데이터\,건수}
$

- 이진 분류의 경우 데이터의 구성에 따라 ML 모델의 성능을 왜곡할 수 있음  
$\rightarrow$ 정확도 수치 하나만 가지고 성능을 평가하지 않음

<br>

## 1.1 타이타닉 - 성별만을 이용한 예측 정확도

- Sex 피처가 1이면 0, 그렇지 않으면 1로 예측하는 단순한 Classifier 생성

In [1]:
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator) :
    
    # fit() 메서드 : 아무것도 학습하지 않음
    def fit(self, X, y=None) :
        pass
    
    # predict() 메서드 : 단순히 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측함
    def predict(self, X) :
        pred = np.zeros((X.shape[0], 1))
        for i in range(X.shape[0]) :
            if X['Sex'].iloc[i] == 1 :
                pred[i] = 0
            else :
                pred[i] = 1
                
        return pred

<br>

`MyDummyClassifier`를 이용해 타이타닉 생존자 예측을 수행

In [2]:
# Null 처리 함수
def fillna(df) :
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df) :
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    
    return df

# 레이블 인코딩 수행
from sklearn.preprocessing import LabelEncoder

def format_features(df) :
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    
    for feature in features :
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
        
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df) :
    
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    
    return df

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할
titanic_df = pd.read_csv('./data/titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df,
                                                    test_size=0.2, random_state=00)

# 위에서 생성한 Dummy Classifier를 이용해 학습/예측/평가 수행
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)

mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도 : {0:.4f}'.format(accuracy_score(y_test, mypredictions)))

Dummy Classifier의 정확도 : 0.7877


$\rightarrow$ 단순한 알고리즘으로 예측을 하더라도 78.77%로 높은 정확도를 나타냄

<br>

## 1.2 불균형한 레이블 값 분포

- 정확도는 **불균형(imbalanced)** 레이블 값 분포에서 ML 모델의 성능을 판단할 경우, 적합한 평가 지표가 아님
- ex) 100개의 데이터 중 90개의 데이터 레이블이 0, 10개의 데이터 레이블이 1인 경우  
$\rightarrow$ 무조건 0으로 예측 결과를 반환하는 ML 모델의 경우라도 정확도가 90%가 됨

<br>

### 1.2.1 불균형한 데이터 세트 : MNIST

- MNIST 데이터 세트를 불균형한 데이터 세트로 변환한 뒤 정확도 지표 적용 시 발생하는 문제점 확인  
  
  
- MNIST 데이터 세트에서 레이블 값이 7인 것만 True, 나머지 값(0~6,8,9)은 모두 False로 변환 (이진 분류 문제로 바꿈)  
$\rightarrow$ 전체 데이터의 10%만 True, 나머지 90%는 False인 불균형한 데이터 세트로 변형됨

<div style='text-align: left; margin-left: 20px;'>
    <img src='./images/Ch03/01/img001.jpg' width='700px'/>
</div>

- 불균형한 데이터 세트에 모든 데이터를 False로 예측하는 Classifier를 이용해 정확도를 측정  
$\rightarrow$ 약 90%에 가까운 예측 정확도를 나타냄

- 정확도 평가 지표의 맹점  
: 무조건 특정한 결과로 찍어도 데이터 분포도가 균일하지 않은 경우 높은 수치가 나타날 수 있는 것

In [5]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator) :
    
    def fit(self, X, y) :
        pass
    
    # 입력값으로 들어오는 X 데이터 세트의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self, X) :
        return np.zeros((len(X), 1), dtype=bool)
    
# 사이킷런 내장 데이터 세트인 load_digits()를 이용해 MNIST 데이터 로딩
digits = load_digits()


# digits 번호가 7번이면 True이고 이를 astype(int)로 1로 변환, 7번이 아니면 False이고 0으로 변환
y = (digits.target == 7).astype(int)
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=11)

<br>

불균형한 데이터로 생성한 `y_test`의 데이터 분포도 확인하고 `MyFakeClassifier`를 이용해 예측과 평가 수행

In [6]:
# 불균형한 레이블 데이터 분포도 확인
print('레이블 테스트 세트 크기 : ', y_test.shape)
print('테스트 세트 레이블 0 과 1의 분포도')
print(pd.Series(y_test).value_counts())

# Dummy Classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)

fakepred = fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는 : {:.3f}'.format(accuracy_score(y_test, fakepred)))

레이블 테스트 세트 크기 :  (450,)
테스트 세트 레이블 0 과 1의 분포도
0    405
1     45
dtype: int64
모든 예측을 0으로 하여도 정확도는 : 0.900


$\rightarrow$ `predict()`의 결과를 `np.zeros()`로 모두 0 값으로 반환함에도 불구하고 450개의 테스트 데이터 세트에 수행한 예측 정확도는 90%

<br>

### 1.2.2 불균형한 데이터 세트에서의 평가 지표

- 정확도 평가 지표는 불균형한 레이블 데이터 세트에서는 성능 수치로 사용돼서는 안됨
- 정확도가 가지는 분류 평가 지표로서 이러한 한계점 극복을 위해 여러 가지 분류 지표와 함께 적용해야 함